In [6]:
import openai
import pandas as pd

OPEN_API_KEY = ''

openai.api_key = OPEN_API_KEY

def get_completion(paragraph, level):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": "[역할] 당신은 초등학교 국어 선생님입니다. 해당 지문을 확인한 후 4지선다 문제를 출제하세요."},
                {"role": "user", "content": f"[방법]"
                                            f" 아래 주어진 지문 내용을 바탕으로 {level} 수준의 4지선다 형태의 객관식 문제 1개를 출제하세요. 반드시 아래 Output의 유형에 맞는 구성으로 만드세요. 요구한 결과물만 출력해줘"
                                            f"[Input] "
                                            f"[지문]  "
                                            f"{paragraph} "
                                            f"[난이도] - {level}"
                                            f""
                                            f"[Output]"
                                            f"{example}"}
            ],
            model="gpt-4o-mini"
        )
        # print(response)
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

news = pd.read_csv("../data/processed/news/result/news_level.csv", sep=',')

example = """
{
    "question": "지문의 내용과 일치하지 않는 것은?",
    "options":  {
        "1":"보기A",
        "2":"보기b",
        "3":"보기c",
        "4":"보기d"
        },
    "answer": 4,
    "reason": "답에 대한 이유"
}
"""

responses = []
level_mapping = {
    1: "미취학아동",
    2: "초등학교 1학년",
    3: "초등학교 2학년",
    4: "초등학교 3학년",
    5: "초등학교 4학년",
    6: "초등학교 5학년",
    7: "초등학교 6학년"
}

for i, row in news.iterrows():
    if (i+1)%50==0:
        print(i+1)
    education_level = level_mapping.get(row['Difficulty_Level'], "Unknown")
    response = get_completion(row['preprocessed_content'], education_level)
    
    if response:
        # print(response)
        responses.append(response)
    else:
        responses.append("Failed to get a response from the API.")
        
# print(responses)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850


In [7]:
responses_df = pd.DataFrame(news[['URL','preprocessed_content', 'Difficulty_Level']])
responses_df['response'] = responses

responses_df.to_csv("../data/processed/news/dt02_22.csv", index=False, encoding='utf-8-sig')

In [1]:
#불러오기
import pandas as pd
import json
from datetime import datetime

responses_df = pd.read_csv("../data/processed/news/dt02_22.csv", sep=',')
news = pd.read_csv("../data/processed/news/result/news_level.csv", sep=',')
content_to_textID = dict(zip(news['preprocessed_content'], news['textID']))

print(responses_df['response'].head())
transformed_data = []

def get_textID(content):
    return content_to_textID.get(content, None)

def transform_row(row):
    response_json = json.loads(row['response'])
    options = response_json.get('options', {})

    answer_key = str(response_json.get("answer", ""))  
    answer_value = ""
    for k, v in options.items():
        if str(k) == answer_key:
            answer_value = v  
            break
            
    now = datetime.now().isoformat()
    transformed = {
        "title": response_json.get("question", ""),
        "content": row['preprocessed_content'],
        "answer" : answer_value,
        "solution": response_json.get("reason", ""),
        "questionTypeCode": "QT01",
        "problemLevelCode": f"PL0{row['Difficulty_Level']}",
        "problemTypeCode": "PT02",
        "problemTypeDetailCode": "PT0222",
        "metadata": {
            "options": [v for v in options.values()] 
        },
        "createdDate": now,
        "modifiedDate": now
    }
    
    return transformed


for i, row in responses_df.iterrows():
    try:
        json.loads(row['response'])  # Try parsing the original JSON
        transformed_data.append(transform_row(row))
    except json.JSONDecodeError as e:
        print(f"Error in row {i}: {e}")
        print(f"Problematic JSON: {row['response']}")

    
dt02_22_df = pd.DataFrame(transformed_data)

0    {\n    "question": "지문의 내용과 일치하지 않는 것은?",\n   ...
1    {\n    "question": "추석 명절에 주의해야 할 것은 무엇인가요?",\...
2    {\n    "question": "지문의 내용과 일치하지 않는 것은?",\n   ...
3    {\n    "question": "이번 파리 패럴림픽에서 종합 1위를 차지한 나라...
4    {\n    "question": "지문의 내용과 일치하지 않는 것은?",\n   ...
Name: response, dtype: object
Error in row 74: Expecting property name enclosed in double quotes: line 8 column 5 (char 145)
Problematic JSON: {
    "question": "칼 라거펠트는 어떤 동물에게 유산을 남겼나요?",
    "options":  {
        "1":"반려견",
        "2":"반려묘",
        "3":"피부미용사",
        "4":"소",
    },
    "answer": 2,
    "reason": "칼 라거펠트는 반려묘 '슈페트'에게 유산을 남겼다고 했기 때문에 정답은 보기 2입니다."
}
Error in row 423: Expecting property name enclosed in double quotes: line 8 column 5 (char 159)
Problematic JSON: {
    "question": "송 교장 선생님이 가장 중요하게 생각하는 것은 무엇인가요?",
    "options":  {
        "1":"학생들의 행복",
        "2":"학교 건물",
        "3":"운동장",
        "4":"선생님들",
    },
    "answer": 1,
    "reason": "송 교장 선생님은 학생들의 행복이 가장 중요하다고 말씀하셨기 때문에, 정답은

In [2]:
print(dt02_22_df)
with open("../data/processed/problem_data/pt0222.json", 'w', encoding='utf-8') as f:
    json.dump(transformed_data, f, indent=4, ensure_ascii=False)
dt02_22_df.to_csv("../data/processed/problem_data/pt0222.csv", index=False, encoding='utf-8-sig')

                                title  \
0                 지문의 내용과 일치하지 않는 것은?   
1             추석 명절에 주의해야 할 것은 무엇인가요?   
2                 지문의 내용과 일치하지 않는 것은?   
3        이번 파리 패럴림픽에서 종합 1위를 차지한 나라는?   
4                 지문의 내용과 일치하지 않는 것은?   
..                                ...   
872               지문의 내용과 일치하지 않는 것은?   
873               지문의 내용과 일치하지 않는 것은?   
874               지문의 내용과 일치하지 않는 것은?   
875        버츄얼 아이돌의 인기가 있는 이유는 무엇인가요?   
876  정부가 케냐에서 오징어를 잡으려는 주된 이유는 무엇인가요?   

                                               content  \
0    푸른 호수 위에 노란 왕관을 쓴 거대한 랍스터가 둥실둥실 떠있어요. 커다란 집게발이...   
1    추석이 코앞으로 다가왔어요. 추석 때는 맛있는 전과 같은 요리를 준비하며 즐거운 연...   
2    활활 타오르는 태양이 내뿜는 빛은 지구로 쏟아져요. 이런 태양 빛을 모아 전기를 만...   
3    지난달 29일부터 열린 2024 파리 패럴림픽이 12일간의 여정을 마감하고, 9일 ...   
4    오륜은 파란색·검은색·빨간색·노란색·초록색의 크기가 같은 다섯 색깔의 고리를 말해요...   
..                                                 ...   
872  머지않아 '배우'라는 직업이 인공지능(AI)으로 대체될지도 모르겠어요. AI로 영상...   
873  정부가 업무개시명령을 이행하지 않고 근무 현장을 이탈한 전공의 555